# Modbus class analysis 

In [ ]:
# sends out registers with high and low byte
def sendDiagnostic(self, register_address, data):
    function_code = 8
    # length of data
    length = 2
    # add values to structure
    packet = pack('>BBHBi', self.address, function_code, register_address, length, data)
    # calculate the crc based upon incoming data
    crc = self.calculateCRC(packet)
    # add bytes to end of packet
    packet += crc
    # write packet to serial
    self.ser.write(packet)
    # set the last requested address to the register address
    self.lastRequestedAddress = register_address


function code 8:
- used to send diagnostics "08 (0x08) Diagnostics (Serial Line only)"
- seems to be used for sending something over serial..

pack
- from struc lib
- packs non byte data into bytes 
- first arg '>BBHBi' is the string format. 
- there is a table that shows what chars correspond to each format
- in this case. 
    - '>' is big-endian and '<' is little-endian, according to chat: When '>' (big-endian) is used, it means that the most significant byte (the "big end") of the data is placed at the lowest memory address.
When '<' (little-endian) is used, it means that the least significant byte (the "little end") of the data is placed at the lowest memory address. (look into this more later)
    - B = unsigned char,  
    - H = unsigned short
    - i = int

crc
- calculates the checksum, used to determine if data is corrupt, if the crc is wrong your messaeg will not be read by driver
- we add the crc to the end of the packet

ser.write
- serial write the packet

self.lastRequestedAddress = register_address
- we save the last address we requested from the motor driver, for some later use...




- remaining questions: how does one find what format you need? It is probably in the data sheet but still. 
- generally how does one go about putting the packet together? is it really just see what is being sent by the motor driver software with a serial monitor and reverse engineer it?






In [ ]:
# sends out registers with 1 byte
def sendSingle(self, register_address, data):
    # the modbus packet function code
    function_code = 6
    # length of data
    length = 2
    # add values to structure
    packet = pack('>BBHH', self.address, function_code, register_address, data)
    # calculate the crc based upon incoming data
    crc = self.calculateCRC(packet)
    # add bytes to end of packet
    packet += crc
    # write packet to serial
    self.ser.write(packet)
    # set the last requested address to the register address
    self.lastRequestedAddress = register_address


function code 6 
- just means write a single holding register with a new value


- remaining questiions:
- how does one know if you need to write a single register with a new value vs sending a diagnostic? 

In [ ]:
def readRegister(self, register_address):
    # the modbus packet function code
    function_code = 3
    # length of data
    length = 2
    # add values to structure
    packet = pack('>BBHH', self.address, function_code, register_address, length)
    # calculate the crc based upon incoming data
    crc = self.calculateCRC(packet)
    # add bytes to end of packet
    packet += crc
    # write packet to serial
    self.ser.write(packet)
    # set the last requested address to the register address
    self.lastRequestedAddress = register_address


function code 3
- code 3 = "read holding register"
- holding register = 16 bit register that you can read or write to. 
- the rest of this function is similar to the previous two, only diff is the function code, the pack format, and the addresses


- questions
- do we really just use the saleae to find the addresses we need to write to? 

In [ ]:
# calculates crcs for modbus packets
def calculateCRC(self, data):
    crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)
    crc_value = crc16(data)
    return crc_value.to_bytes(2, byteorder='little')  # Use 'little' byte order


calculatingCRC uses some algo to calc, 

- questions
- how does one find out what args are needed for the crcmod.mkCrcF()?



In [ ]:
# returns a single byte if data is in the rx buffer
def readByte(self):
    if self.ser.is_open:
        if self.ser.in_waiting > 0:
            # if there is waiting the be read add it into the local software buffer
            readByte = self.ser.read(size=1)
            self.serialBuffer.append(readByte)
            return readByte
    return False


in_waiting
- gets the number of bytes in the input buffer

read
- of course reads from ser, size=1 is just one byte to read

serialbuffer
- this is a varin the constructor of this class and is not related to modbus lib
- however we are still using it to add bytes we read into it as a buffer






In [ ]:

def processPackets(self):
    '''call this as often as possible to process incoming 
    packets from the serial buffer.  Will return true if a packet was 
    routed'''

    # calling this, processes incoming bytes into a local buffer
    self.readByte()

    # only check for a valid packet if data is in the buffer
    if len(self.serialBuffer) >= 2:

        # fetch the first byte in the serial buffer
        address_in = unpack('B', self.serialBuffer[0])[0]

        # get the function code
        function_code = unpack('B', self.serialBuffer[1])[0]

        # set the packet length based upon function code
        returnPacketLength = 0
        if function_code == 8:
            returnPacketLength = 11
        elif function_code == 3:
            returnPacketLength = 9
        elif function_code == 6:
            returnPacketLength = 9

        # check to see if the first byte is the start character '$'
        if address_in != self.address:
            print("Bad Address:", self.serialBuffer[0])
            # if packet is not valid remove data from buffer
            del self.serialBuffer[0]

        # if address of packet was present look for a length byte
        elif len(self.serialBuffer) >= returnPacketLength:
            # get the length
            length = unpack('B', self.serialBuffer[2])[0]
            # get the address
            registerAddress = unpack('>H', b''.join(self.serialBuffer[3:5]))[0]
            # get the data
            self.rx_data = unpack('>i', b''.join(self.serialBuffer[3:7]))[0]
            # get the crc
            crc = unpack('<H', b''.join(self.serialBuffer[returnPacketLength - 2:returnPacketLength]))[0]

            # calculate the CRC based upon the packet data
            calculated_crc = self.calculateCRC(b''.join(self.serialBuffer[0:returnPacketLength - 2]))
            # convert calculated crc to int
            calculated_crc = unpack('<H', calculated_crc)[0]

            if MOD_BUS_DEBUG == True:
                print("-------------------")
                print("Modbus - Packet Bytes: ", self.serialBuffer[0:returnPacketLength])
                print("Modbus - Total Bytes In Buffer: ", len(self.serialBuffer))
                print("Modbus - Function Code: ", function_code)
                print("Modbus - Register Address: ", registerAddress)
                print("Modbus - Length Byte: ", length)
                print("Modbus - Data Payload: ", self.rx_data)
                print("Modbus - CRC: ", crc)
                print("Modbus - Calculated CRC: ", calculated_crc)

            if calculated_crc == crc:
                # remove packet from buffer
                del self.serialBuffer[0:returnPacketLength]

                # return true if there is a packet to read
                return True
            # if it was a bad crc delete the packet without routing the payload
            else:
                del self.serialBuffer[0:returnPacketLength]
                print("bad crc %i" % calculated_crc)

    # return false if there is no packet to read
    return False


- ok we start by calling our read byte function makes sense
- if the buffer is >= 2 we care, a single byte is not a valid packet. makes sense, even a small packet will have at elast two bytes, the data and the crc.
 

In [ ]:
# fetch the first byte in the serial buffer
    address_in = unpack('B', self.serialBuffer[0])[0]

unpack
- part of struc of course
- 'B' is unsigned char
- we pass in the first byte from the serial buffer so that we can unpack it.
- the result is a list where the first el is the address, and the second el is the function code  hence the following line 

function_code = unpack('B', self.serialBuffer[1])[0]

In [ ]:
# set the packet length based upon function code
    returnPacketLength = 0
    if function_code == 8:
        returnPacketLength = 11
    elif function_code == 3:
        returnPacketLength = 9
    elif function_code == 6:
        returnPacketLength = 9


- so we see 3 possible function codes, 8, 3, 6 which are the codes we saw above
- different lengths explained by gemini: " different Modbus function codes have different response packet structures and data lengths." it basically says use the modbus org to see what length you need...

- questions:
- not sure about the return packet length, as in, why is it a different int and how do we figure out that len

In [ ]:
# check to see if the first byte is the start character '$'
if address_in != self.address:
    print("Bad Address:", self.serialBuffer[0])
    # if packet is not valid remove data from buffer
    del self.serialBuffer[0]


- pretty straightforward, self.address is the ID of our lift motor, so if the ID is not that of our lift motor then the packet is 100% broken since we are only talking to one motor.

In [ ]:

    # if address of packet was present look for a length byte
    elif len(self.serialBuffer) >= returnPacketLength:
        # get the length
        length = unpack('B', self.serialBuffer[2])[0]
        # get the address
        registerAddress = unpack('>H', b''.join(self.serialBuffer[3:5]))[0]
        # get the data
        self.rx_data = unpack('>i', b''.join(self.serialBuffer[3:7]))[0]
        # get the crc
        crc = unpack('<H', b''.join(self.serialBuffer[returnPacketLength - 2:returnPacketLength]))[0]

        # calculate the CRC based upon the packet data
        calculated_crc = self.calculateCRC(b''.join(self.serialBuffer[0:returnPacketLength - 2]))
        # convert calculated crc to int
        calculated_crc = unpack('<H', calculated_crc)[0]

        if MOD_BUS_DEBUG == True:
            print("-------------------")
            print("Modbus - Packet Bytes: ", self.serialBuffer[0:returnPacketLength])
            print("Modbus - Total Bytes In Buffer: ", len(self.serialBuffer))
            print("Modbus - Function Code: ", function_code)
            print("Modbus - Register Address: ", registerAddress)
            print("Modbus - Length Byte: ", length)
            print("Modbus - Data Payload: ", self.rx_data)
            print("Modbus - CRC: ", crc)
            print("Modbus - Calculated CRC: ", calculated_crc)

        if calculated_crc == crc:
            # remove packet from buffer
            del self.serialBuffer[0:returnPacketLength]

            # return true if there is a packet to read
            return True
        # if it was a bad crc delete the packet without routing the payload
        else:
            del self.serialBuffer[0:returnPacketLength]
            print("bad crc %i" % calculated_crc)

# return false if there is no packet to read
    return False


- we get the len, address, data, and crc from the valid message
- we calculate the crc so that we can compare the calculated crc with the read crc
- if calc != read, we have an invalid message
- then there are debug messages that can help you see what is in the message

- questions
- why does this need to be true "len(self.serialBuffer) >= returnPacketLength"


In [ ]:
if calculated_crc == crc:
    # remove packet from buffer
    del self.serialBuffer[0:returnPacketLength]

    # return true if there is a packet to read
    return True
# if it was a bad crc delete the packet without routing the payload
else:
    del self.serialBuffer[0:returnPacketLength]
    print("bad crc %i" % calculated_crc)


now we get to the good part
- "if calculated_crc == crc" this checks if the packet was valid, if it is not we del it from the buffer and print that it was a bad crc makes perfect sense
- using 'del' on a list is something i somehow have not seen til now. i used pop.
- del is better since it removes things in place and you can remove slices.
- the reason we have 'returnPacketLength' is because it is used for the upper limit on the slice we will perform to remove the received packet from the buffer
